## Рекомендательные системы
### Урок 2. Бейзлайны и детерминированные алгоритмы item-item

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k
import re

In [4]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [16]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(np.arange(len(items_weights)), 
                            size=n, 
                            p=np.array(items_weights.weight), 
                            replace=False)
    
    return recs.tolist()

In [17]:
%%time
items_weights = pd.DataFrame(data.item_id.value_counts(normalize=True).reset_index().values, 
                             columns=['item_id', 'weight'])

weighted_random_recommendation(items_weights, n=5)

CPU times: user 115 ms, sys: 44.2 ms, total: 159 ms
Wall time: 157 ms


[10724, 8743, 2857, 4637, 16]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [31]:
result = pd.read_csv('./predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [26]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
        
    return np.sum(np.isin(bought_list, recommended_list[:k])) / len(recommended_list[:k])   

In [33]:
for col in result.columns[1:]:
    result[col] = [np.array(re.findall('\d+', x), dtype=np.uintc) for x in result[col]]

In [34]:
# TEST
fact = result['actual']
for col in result.columns[2:]:
    lr = []
    for rec, act in zip(result[col], fact):
        lr.append(precision_at_k(rec, act, k=5))
    prec=pd.Series(lr).mean()
    print(f'Precistion@K {col} = {prec}')

Precistion@K random_recommendation = 0.0005876591576885408
Precistion@K popular_recommendation = 0.15523996082272282
Precistion@K itemitem = 0.03359451518119491
Precistion@K cosine = 0.03525954946131244
Precistion@K tfidf = 0.036141038197845254
Precistion@K own_purchases = 0.17998694090760692


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [35]:
# your_code